In [1]:
import numpy as np
from datetime import datetime
import os

### Just random testing stuff

In [11]:
now = datetime.now().strftime("%Y-%m-%d_%Hh-%Mm-%Ss")
print(now)

outf = os.path.join(os.getcwd(), 'figures', 'sim_'+now+'.mp4')
print(outf)

2023-09-23_12h-14m-33s
/home/janmak98/Documents/masterthesis/chemokine/monte_carlo/figures/sim_2023-09-23_12h-14m-33s.mp4


In [101]:

particle_diffusivity = [1, 2, 3,4, 5]
friction = np.array(particle_diffusivity)**0.7
# print(friction)
# print(particle_diffusivity)
# print(np.array(particle_diffusivity))
# uno = np.array([1,1,1], ndmin=2).T
# print(uno)
# prob = np.append(uno, friction.T, axis=1)
# prob = np.append(prob, friction.T, axis=1)
# print(prob)
# print(np.array([np.array([1,1,1], ndmin=2).T, np.array(friction, ndmin=2).T, friction]).T)
probabilities = np.array([np.array([1,1,1,1,1]), np.array(friction), np.array(friction)]).T
#probabilities = np.array([[1,1,1], particle_diffusivity, particle_diffusivity]).T
print(probabilities)
probabilities = probabilities/np.sum(probabilities, axis=1, keepdims=True)

print(probabilities)
step_choice = np.array([0, 1, -1])
n_particles_0 = 10

step = np.random.choice(step_choice, size=(n_particles_0, 2), p=probabilities[0])

print(probabilities[0])
# print(probabilities[1].shape)
# print(np.sum(probabilities[1]))
print(step)

[[1.         1.         1.        ]
 [1.         1.62450479 1.62450479]
 [1.         2.15766928 2.15766928]
 [1.         2.63901582 2.63901582]
 [1.         3.08516931 3.08516931]]
[[0.33333333 0.33333333 0.33333333]
 [0.23534896 0.38232552 0.38232552]
 [0.18813477 0.40593261 0.40593261]
 [0.15928559 0.4203572  0.4203572 ]
 [0.13946343 0.43026829 0.43026829]]
[0.33333333 0.33333333 0.33333333]
[[ 0  1]
 [ 1  0]
 [-1  1]
 [ 1 -1]
 [-1  0]
 [ 1  1]
 [ 1  0]
 [ 0 -1]
 [ 1  0]
 [ 1  0]]


In [1]:
pos = np.random.randint(0, 10, size=(10, 2))

print(pos)
collagen_y = 5

idx_fixed = np.where(pos[:, 1] == collagen_y)[0]
print(idx_fixed)

print(pos[idx_fixed])
print(idx_fixed.shape[0])

NameError: name 'np' is not defined

In [ ]:
grid_size = [10, 12]
n_particles_3 = 5

pos_3x = np.random.randint(0, grid_size[0], size=(n_particles_3))
pos_3y = np.random.randint(0, grid_size[1], size=(n_particles_3))
pos_3 = np.array([pos_3x, pos_3y]).T
print(pos_3)

[[ 9  1]
 [ 0  7]
 [ 2 11]
 [ 9  5]
 [ 6  6]]


In [6]:
a = 1 
b = 2

if a==1 and b==2:
    print('yes')
elif a==1:
    print('1')
elif b==2:
    print('2')
else:
    print('no')

yes


# Compute hopping probabilities with neighbor particle present

In [4]:
# from physical_units_diffusion import *

In [9]:
def calc_time_step_ratio(energy_matrix, diffusivity_array):
    # compute ratio of time step to grid constant squared Delta/a^2 by setting the maximum movement probability to be 1/4
    if len(diffusivity_array) != len(energy_matrix):
        print("Error: length of diffusivity array must be the same as the number of rows in the energy matrix")
        return
    max_dexp = np.max(np.exp(energy_matrix) * diffusivity_array) # since energy matrix is symmetric, it is sufficient to multiply the diffusivity array, such that D1 exp(E12) and D2 exp(E12) = D2 exp(E21) are both considered
    ratio = 1 / (5 * max_dexp)
    return ratio

# function that takes  thye ratio of time step to grid constant squared Delta/a^2 in [s/m^2] and a diffusivity in [m^2/s] and returns the probability of a particle to move in one direction and the probability to stay in the same position
def calc_probabilities_free(ratio, diffusivity_array):
    p_move = diffusivity_array * ratio * 1.25
    p_stay = 1 - 4 * p_move
    return p_move, p_stay

def calc_probabilities_with_neighbor(energy_matrix, diffusivity_array):
    ratio = calc_time_step_ratio(energy_matrix, diffusivity_array)
    p_move_free = calc_probabilities_free(ratio, diffusivity_array)[0]
    p_move_neighbor = np.zeros(int(len(energy_matrix) * (len(energy_matrix) - 1) / 2))
    for i in range(len(energy_matrix)):
        for j in range(i+1, len(energy_matrix)):
            p_move_neighbor[i + j - 1] = p_move_free[i] * np.exp(energy_matrix[i, j])
            print(i + j - 1, i, j)
    return p_move_neighbor

In [10]:
energy_matrix = np.array([[0, 1, 2], [1, 0, 3], [2, 3, 0]])
diffusivity_array = np.array([1, 2, 3])
calc_probabilities_with_neighbor(energy_matrix, diffusivity_array)

0 0 1
1 0 2
2 1 2


array([0.01127794, 0.03065662, 0.16666667])